In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pandas as pd
import cv2
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#Define the ideal picture size for training
size = 100

# Import image files of 3D holes
rectHole = cv2.imread("new3D/HOLE/rectangleHole.png", 0)
rectHole = np.invert(rectHole)

cylinderHole = cv2.imread('new3D/HOLE/cylinderHole.png', 0)
cylinderHole = np.invert(cylinderHole)

octagonHole = cv2.imread('new3D/HOLE/octagonHole.png', 0)
octagonHole = np.invert(octagonHole)

print("done imports")

In [ ]:
def recenterObject(image):
    #### find the center of image
    original_height, original_width = image.shape
    og_ch = original_height / 2
    og_cw = original_width / 2

    # calculate moments of binary image
    M = cv2.moments(image)

    # calculate x,y coordinate of center
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])

    ### find shift size
    offsetX = og_cw - cx
    offsetY = og_ch - cy

    # recenter object to the center of the image
    TM = np.float32([[1, 0, offsetX], [0, 1, offsetY]])

    return cv2.warpAffine(image, TM, (original_width, original_height))

print("done recenterObject")

In [ ]:
def rotateObject(image):
    # set threshold for contours
    ret, thresholdedImage = cv2.threshold(image, 127, 255, 0)
    
    #get external contour of the object
    contour, hierarchy = cv2.findContours(thresholdedImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    external = cv2.convexHull(contour[0])
    
    # find the minimum rectangle which encompases the object (to determine degrees of rotation)
    boundryBox = cv2.minAreaRect(external)
    
    # retrieve the corners of the rectangle
    box = cv2.boxPoints(boundryBox)
    box = np.int0(box)
    
    # find the angle of rotation
    # tilted right
    if  box[0][0] < box[1][0]:
        if boundryBox[-1] < 45:
            angle = boundryBox[-1]
        else:
            angle = boundryBox[-1] - 90
            
    # upright
    elif boundryBox[-1] == 0.0:
        angle=0
        
    # tilted left
    else:
        angle = boundryBox[-1] - 90
    
    # find center of the image
    original_height, original_width = image.shape
    center = (original_width // 2, original_height // 2)
    
    # rotate the image respectively to the angle via rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (original_width, original_height), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

print("done Rotating")

In [ ]:
def checkFitNegativePixels(hole, image):
    original_height, original_width = hole.shape

    for i in range(original_height):
        for j in range(original_width):
            pixel_hole = hole[i, j]
            pixel_image = image[i, j]
            if (int(pixel_hole) == 0) and (int(pixel_image) > 0):
                return 0
    return 1

print("done checkFitNegativePixels")

In [ ]:
def categorize(object):
    category = []
    
    category.append(checkFitNegativePixels(rectHoleCentered, object))
    category.append(checkFitNegativePixels(cylinderHoleCentered, object))
    category.append(checkFitNegativePixels(octagonHoleCentered, object))

    return category

print("done categorize")

In [ ]:
# Recenter HOLES
rectHoleCentered = recenterObject(rectHole)
cylinderHoleCentered = recenterObject(cylinderHole)
octagonHoleCentered = recenterObject(octagonHole)

print('done recentering  holes!')

In [ ]:
image1= cv2.imread("new3D/objects/rectPrismRot16.png", 0)

print("original")
plt.imshow(image1)
plt.show()

inv_image = np.invert(image1)
image1 = rotateObject(inv_image)
image1 = recenterObject(image1)

print("modified object")
plt.imshow(image1)
plt.show()

print("images of the holes")
plt.imshow(rectHoleCentered)
plt.show()
plt.imshow(cylinderHoleCentered)
plt.show()
plt.imshow(octagonHoleCentered)
plt.show()

print("categorize",categorize(image1))

In [ ]:
training_data = []


def labelImages():
    counter=0
    for img in os.listdir('new3D/objects'):
        
        image = cv2.imread(os.path.join('new3D/objects', img), 0)
        
        print(img)

        inv_image = np.invert(image)
        object = rotateObject(inv_image)
        object = recenterObject(object)
        
        #resize original object image to train size
        image = cv2.resize(image, (size, size))
        
        #add to the training data
        training_data.append([image, categorize(object)])
        
        counter=counter+1
    print(counter)
    
    return training_data

labelImages()
print("done labelling training data")

In [ ]:
file = open("RAW3D_training_data", "wb")
pickle.dump(training_data, file)
file.close

In [2]:
retrieve_training_data= open("RAW3D_training_data", "rb")
training_data = pickle.load(retrieve_training_data)
print("done pickle out!")

done pickle out!


In [3]:
X=[]
Y=[]

for object_image, category in training_data:
    X.append(object_image)
    Y.append(category)


X= np.array(X).reshape(-1, size, size, 1)
Y= np.array(Y)

print("X shape",X.shape)
print("Y shape",Y.shape)

X shape (404, 100, 100, 1)
Y shape (404, 3)


In [4]:
# create individual label for each shape model

def shapeCategory(Y, shape):

    if shape=='rectanglePrism':
        shape=0
    elif shape=='cylinder':
        shape=1
    elif shape=='octogon':
        shape=2
    else:
        shape=3
        print("error")
        
    shapeCategoryLabel=[]

    for i in range(len(Y)):
        shapeCategoryLabel.append(Y[i][shape])
    shapeCategoryLabel= np.array(shapeCategoryLabel)
    return shapeCategoryLabel
print("done!")

done!


In [ ]:
# define label for each shape

rectanglePrismLabels=shapeCategory(Y, "rectanglePrism")
cylinderLabels=shapeCategory(Y, "cylinder")
octogonLabels=shapeCategory(Y, "octogon")

In [6]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

class shapeCategoryModel():

    def skeleton(self, inputs):
        
        x = Conv2D(16, (3, 3))(inputs)
        x = Activation("relu")(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
 
        x = Conv2D(16, (3, 3))(x)
        x = Activation("relu")(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
        x=Flatten()(x)
        
        x=Dense(16)(x)    
        x=Dense(1)(x)
        
        x = Activation("sigmoid")(x)
              
        return x
    
    def voltran(self):

            input_shape = (size, size, 1)
            inputs = Input(shape=input_shape)
        
            rectanglePrism = self.skeleton(inputs)
            cylinder = self.skeleton(inputs)
            octogon = self.skeleton(inputs)

            modelshapeCategory = Model(inputs=inputs,
                         outputs = [rectanglePrism, cylinder, octogon],
                         name="shapeFitting")
            return modelshapeCategory

In [7]:
kfold = KFold(n_splits=5, shuffle=True) #, shuffle=True
K_value=1
for train, test in kfold.split(X, Y):
    modelshapeCategory = shapeCategoryModel().voltran()


    modelshapeCategory.compile(
            loss="binary_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])
#     metrics=["accuracy", "mse"])
    
    history = modelshapeCategory.fit(X, [rectanglePrismLabels, cylinderLabels, octogonLabels], batch_size=25, epochs=10, verbose=0)
    
    scores=modelshapeCategory.evaluate(X, [rectanglePrismLabels, cylinderLabels, octogonLabels], verbose=0)
       
    rectangleAccuracy=scores[4]
    cylinderAccuracy=scores[5]
    octogonAccuracy=scores[6]
    
    print("Results of K:",K_value,"rectangleAccuracy:","{:.2f}".format(rectangleAccuracy),
          "cylinderAccuracy:","{:.2f}".format(cylinderAccuracy),
          "octogonAccuracy:","{:.2f}".format(octogonAccuracy))
          
    K_value+=1


Results of K: 1 rectangleAccuracy: 0.95 cylinderAccuracy: 0.97 octogonAccuracy: 0.78
Results of K: 2 rectangleAccuracy: 1.00 cylinderAccuracy: 1.00 octogonAccuracy: 0.82
Results of K: 3 rectangleAccuracy: 1.00 cylinderAccuracy: 0.97 octogonAccuracy: 0.98
Results of K: 4 rectangleAccuracy: 0.97 cylinderAccuracy: 1.00 octogonAccuracy: 0.88
Results of K: 5 rectangleAccuracy: 0.78 cylinderAccuracy: 0.82 octogonAccuracy: 1.00


In [ ]:
rectResults = []
cylinderResults = []
octogonResults = []

def roundVal(x):
    if x <.5:
        return 0
    else:
        return 1
    
for img in os.listdir('new3D/objects'):
        
        image = cv2.imread(os.path.join('new3D/objects', img), 0)
        resizedImage = cv2.resize(image, (size, size))

        npImage=np.array(resizedImage).reshape(-1, size, size)
        modelresult = modelshapeCategory.predict(npImage)
        
        rectResult=roundVal(modelresult[0][0][0])
        cylinderResult=roundVal(modelresult[1][0][0])
        octogonResult=roundVal(modelresult[2][0][0])
        
        rectResults.append(rectResult)
        cylinderResults.append(cylinderResult)
        octogonResults.append(octogonResult)


In [9]:
confusionRect = metrics.confusion_matrix(rectanglePrismLabels, rectResults)
confusionCylinder = metrics.confusion_matrix(cylinderLabels, cylinderResults)
confusionOctagon = metrics.confusion_matrix(octogonLabels, octogonResults)

print("rect:",confusionRect)
print("cylinder:",confusionCylinder)
print("octagon:",confusionOctagon)

404
404
rect: [[170   8]
 [ 79 147]]
cylinder: [[160  17]
 [ 56 171]]
octagon: [[ 81   2]
 [  0 321]]


In [10]:
TP = confusionRect[1, 1]
TN = confusionRect[0, 0]
FP = confusionRect[0, 1]
FN = confusionRect[1, 0]

print("Classification Accuracy",(TP + TN) / float(TP + TN + FP + FN))

print("Classification Error",(FP + FN) / float(TP + TN + FP + FN))

print("sensitivity",TP / float(FN + TP))

print("precision",TP / float(TP + FP))

Classification Accuracy 0.7846534653465347
Classification Error 0.21534653465346534
sensitivity 0.6504424778761062
precision 0.9483870967741935


In [11]:
TP = confusionCylinder[1, 1]
TN = confusionCylinder[0, 0]
FP = confusionCylinder[0, 1]
FN = confusionCylinder[1, 0]

print("Classification Accuracy",(TP + TN) / float(TP + TN + FP + FN))

print("Classification Error",(FP + FN) / float(TP + TN + FP + FN))

print("sensitivity",TP / float(FN + TP))

print("precision",TP / float(TP + FP))

Classification Accuracy 0.8193069306930693
Classification Error 0.1806930693069307
sensitivity 0.7533039647577092
precision 0.9095744680851063


In [12]:
TP = confusionOctagon[1, 1]
TN = confusionOctagon[0, 0]
FP = confusionOctagon[0, 1]
FN = confusionOctagon[1, 0]

print("Classification Accuracy",(TP + TN) / float(TP + TN + FP + FN))

print("Classification Error",(FP + FN) / float(TP + TN + FP + FN))

print("sensitivity",TP / float(FN + TP))

print("precision",TP / float(TP + FP))

Classification Accuracy 0.995049504950495
Classification Error 0.0049504950495049506
sensitivity 1.0
precision 0.9938080495356038
